<a href="https://colab.research.google.com/github/rana14ce/Dataset-of-the-Multinational-Pile/blob/main/GitHub_XGBoost_with_5_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CTRL+Shift+Space bar = docstring
# CTRL+Space bar = autocomplete

# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
url = 'https://raw.githubusercontent.com/rana14ce/Dataset-of-the-Multinational-Pile/main/Dataset%20of%20the%20Multinational%20Pile.csv'
data = pd.read_csv(url)

In [ ]:
data.head()

,Phi_s,Phi_t,L_m,A_m2,Sv _KN/m2,Qm_KN
0,33.0,38.0,255,24.5,0.1310,2615
1,34.0,37.5,206,19.8,0.2230,3675
2,33.0,38.0,223,21.5,0.1310,2164
3,33.0,37.5,210,20.2,0.1468,3042
4,33.0,37.0,206,19.9,0.1821,2856


In [ ]:
X = data.drop('Qm_KN', axis=1)
y = data['Qm_KN']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the XGBoost Regressor
xgb_model = XGBRegressor(max_depth=5, gamma=0.25)

# Perform 5-Fold Cross-Validation
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
print("5-Fold CV RMSE: ", cv_rmse)
print("Mean CV RMSE: ", np.mean(cv_rmse))


5-Fold CV RMSE:  [1115.29101722  556.38951522 1015.70790878  862.92779986 1028.73072949]
Mean CV RMSE:  915.809394115646


In [ ]:
# Train the XGBoost model on the entire training set
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Test R^2: ", r2)
print("Test RMSE: ", rmse)


Test R^2:  0.2937907163399035
Test RMSE:  1157.1770156209277
